In [ ]:
%pip install -U --pre tensorflow=="2.*"
%pip install tf_slim

In [ ]:
%pip install pycocotools

In [ ]:
import os
import pathlib


if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python3 -m pip install .

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [10]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [11]:
# настроим опции tf1 в `utils.ops`
# tf.compat - модуль совместимости, позволяет писать код, который
# работает, как в TensorFlow 1.x, так и в 2.x.
utils_ops.tf = tf.compat.v1

# настроим путь к файлу gfile
tf.gfile = tf.io.gfile

In [12]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name,
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

In [16]:
# Список строк, которые используются для добавления правильной метки для
# каждого поля
PATH_TO_LABELS = './models/research/object_detection/data/mscoco_label_map.pbtxt'
# models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

In [ ]:
!ls /content/drive/MyDrive/content

In [ ]:
# Если вы хотите протестировать код с вашими изображениями,
# просто добавьте путь к изображениям в TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/drive/MyDrive/content')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpeg")))
TEST_IMAGE_PATHS

In [ ]:
print(detection_model.signatures['serving_default'].inputs)

In [ ]:
detection_model.signatures['serving_default'].output_dtypes

In [ ]:
detection_model.signatures['serving_default'].output_shapes

In [22]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # На входе должен быть тензор,
  # конвертируем image используя `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # модель ожидает пакет изображений,
  # поэтому добавляем ось(измерение)
  # c помощью метода with `tf.newaxis`.

  input_tensor = input_tensor[tf.newaxis,...]

  # запускаем нейросеть для вывода результатов
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # Все выходные данные являются пакетом тензоров пакетов.
  # Преобразуем в массивы numpy и возьмем  индекс [0],
  # чтобы удалить пакетное измерение.
  # Нас интересуют только первое измерение - num_detections.

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy()
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes должен быть целочисленным
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

  # Обработаем модель с помощью масок:
  if 'detection_masks' in output_dict:
    # Изменим формат маски bbox в соответствии
    # с размером изображения.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

  return output_dict

In [23]:
def show_inference(model, image_path):
  # представление изображения на основе массива
  # будет использовано позже для подготовки
  # результирующего изображения с рамками и надписями на них.

  image_np = np.array(Image.open(image_path))
  # Запускаем распознавание
  output_dict = run_inference_for_single_image(model, image_np)
  # Визуализируем результаты распознавания
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)


## Сегментация(выделение) распознанных экземпляров

In [ ]:
model_name = "mask_rcnn_inception_resnet_v2_atrous_coco_2018_01_28"
masking_model = load_model(model_name)

In [ ]:
masking_model.signatures['serving_default'].output_shapes

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(masking_model, image_path)

In [ ]:
!ls -all